## A) Overwrite backend with the Agent (planner/executor/aggregator + traces)

In [2]:
%%writefile Week_7/backend/agent_runtime.py
import math, time, json, uuid, re, csv, io
from pathlib import Path
from typing import Dict, Any, List, Optional, Tuple

# --------------- Guardrails -----------------
UNSAFE_TERMS = {"nsfw", "violent", "illegal", "weapon", "blood", "gore"}
MAX_HOPS     = 4
REQ_TIMEOUT  = 120  # seconds total per /agent call

def blocked_by_guardrails(text: str) -> Optional[str]:
    t = text.lower()
    for bad in UNSAFE_TERMS:
        if bad in t:
            return bad
    return None

# --------------- Tools ----------------------

class Tools:
    """
    Toolkit: ProjectQA (stub), Stable Diffusion, Calculator, CSV Summarizer.
    SD callable must be injected so we don't import diffusers here.
    """
    def __init__(self, sd_callable, negative_prompt: str, size: Tuple[int,int]):
        self.sd_callable = sd_callable
        self.negative_prompt = negative_prompt
        self.size = size  # (W, H)

    def tool_project_qa(self, query: str) -> Tuple[str, List[Dict[str,str]]]:
        # TODO: plug your real Week-6 Graph-RAG / Multi-Hop here
        citations = [
            {"title": "Week 7 Repo", "url": "https://github.com/sudhakarjerribanda/Capston_Handson/tree/main/Week_7"},
            {"title": "Task 1 Colab", "url": "https://colab.research.google.com/drive/1yZd5zUUmNjeDH-_mpVEkNCO4fonsmW69?usp=sharing"}
        ]
        ans = f"(ProjectQA stub) For query: {query}\n- Use Graph-RAG evidence cards, IMRaD blocks, BDI/trust gauge, PHI sanitizer badge."
        return ans, citations

    def tool_sd_generate(self, prompt: str, steps: int = 25, guidance: float = 7.5, fn_prefix: str = "agent") -> str:
        """
        Returns image filename saved under Week_7/diffusion.
        """
        W, H = self.size
        filename = self.sd_callable(
            prompt=prompt,
            negative_prompt=self.negative_prompt,
            steps=steps,
            guidance=guidance,
            height=H,
            width=W,
            filename_prefix=fn_prefix
        )
        return filename

    def tool_calculator(self, expr: str) -> str:
        """
        Very small safe calculator: digits, + - * / ( ) . ^ and spaces only.
        """
        if not re.fullmatch(r"[0-9\.\+\-\*\/\(\)\s\^]+", expr):
            return "Calculator refused: unsupported characters."
        try:
            # implement ^ as ** for exponent
            expr_py = expr.replace("^", "**")
            val = eval(expr_py, {"__builtins__": {}}, {})
            return str(val)
        except Exception as e:
            return f"Calculator error: {e}"

    def tool_csv_summary(self, csv_text: str, top_k: int = 3) -> str:
        """
        Summarize a small CSV (e.g., latency logs).
        """
        try:
            f = io.StringIO(csv_text)
            rows = list(csv.DictReader(f))
            n = len(rows)
            if n == 0:
                return "CSV has no rows."
            cols = rows[0].keys()
            # try to summarize numeric columns by mean
            means = {}
            for c in cols:
                vals = []
                for r in rows:
                    try:
                        vals.append(float(r[c]))
                    except:
                        pass
                if vals:
                    means[c] = sum(vals) / max(1, len(vals))
            head = rows[:top_k]
            return f"Rows: {n}\nMeans: {means}\nHead: {head}"
        except Exception as e:
            return f"CSV summary error: {e}"

# --------------- Planner / Executor / Aggregator ---------------

class Agent:
    def __init__(self, tools: Tools):
        self.tools = tools

    def plan(self, user_input: str, hops_done: int) -> Dict[str, Any]:
        """Very simple heuristic router."""
        text = user_input.lower()
        if any(k in text for k in ["image", "diagram", "illustration", "infographic", "generate", "sd"]):
            return {"tool": "sd", "args": {
                "prompt": user_input + ", flat vector, teal+indigo accents, rounded cards, subtle shadows, clean typography"
            }}
        if "calc:" in text or "calculate" in text or re.search(r"[0-9].*[\+\-\*\/\^].*[0-9]", text):
            # allow "calc: 2+2"
            expr = user_input.split("calc:", 1)[1].strip() if "calc:" in text else user_input
            return {"tool": "calc", "args": {"expr": expr}}
        if "csv:" in text:
            csv_text = user_input.split("csv:", 1)[1]
            return {"tool": "csv", "args": {"csv_text": csv_text}}
        # default → Project QA
        return {"tool": "qa", "args": {"query": user_input}}

    def execute(self, plan: Dict[str, Any]) -> Tuple[str, Optional[str], List[Dict[str,str]]]:
        tool = plan["tool"]
        args = plan.get("args", {})
        if tool == "qa":
            ans, cites = self.tools.tool_project_qa(**args)
            return ans, None, cites
        if tool == "sd":
            fname = self.tools.tool_sd_generate(**args)
            return f"Generated image: {fname}", fname, []
        if tool == "calc":
            out = self.tools.tool_calculator(**args)
            return f"Calculator: {out}", None, []
        if tool == "csv":
            out = self.tools.tool_csv_summary(**args)
            return f"CSV summary: {out}", None, []
        return "Unknown tool.", None, []

    def aggregate(self, final_chunks: List[str]) -> str:
        return "\n---\n".join(final_chunks)

    def run(self, user_input: str, deadline_ts: float) -> Dict[str, Any]:
        """
        Runs up to MAX_HOPS or until deadline; returns trace dict.
        """
        guard = blocked_by_guardrails(user_input)
        if guard:
            return {
                "final": f"Blocked by guardrails: {guard}",
                "hops": [], "citations": [], "image": None, "latency_ms": 0
            }

        hops_trace = []
        chunks = []
        citations: List[Dict[str,str]] = []
        produced_image = None

        for hop in range(MAX_HOPS):
            if time.time() > deadline_ts:
                chunks.append("⏰ Stopped: deadline reached.")
                break
            t0 = time.time()
            plan = self.plan(user_input, hop)
            text, image, cites = self.execute(plan)
            dt = int((time.time() - t0) * 1000)
            hops_trace.append({
                "tool": plan["tool"],
                "input": plan.get("args", {}),
                "output_preview": (image or text)[:200],
                "latency_ms": dt
            })
            if image and not produced_image:
                produced_image = image
            if cites:
                citations.extend(cites)
            chunks.append(text)

            # Simple early stop: if we generated an image or answered QA, stop.
            if plan["tool"] in ("sd", "qa"):
                break

        final = self.aggregate(chunks)
        latency_total = sum(h["latency_ms"] for h in hops_trace)

        return {
            "final": final, "hops": hops_trace, "citations": citations,
            "image": produced_image, "latency_ms": latency_total
        }

# --------------- Persistent trace log ---------------
def append_trace(root: Path, user_input: str, trace: Dict[str, Any]):
    out = root / "screenshots" / "agent_traces.jsonl"
    out.parent.mkdir(parents=True, exist_ok=True)
    row = {
        "id": uuid.uuid4().hex,
        "ts": int(time.time()),
        "input": user_input,
        "trace": trace
    }
    with out.open("a", encoding="utf-8") as f:
        f.write(json.dumps(row) + "\n")


Writing Week_7/backend/agent_runtime.py


FileNotFoundError: [Errno 2] No such file or directory: 'Week_7/backend/agent_runtime.py'

In [4]:
import os

os.makedirs('Week_7/backend', exist_ok=True)

In [5]:
%%writefile Week_7/backend/main.py
import time, os, uuid, json
from pathlib import Path
from typing import List, Optional, Dict
from fastapi import FastAPI, HTTPException, Header
from pydantic import BaseModel, Field
import torch
from PIL import Image

# Local modules
from .settings import settings
from .agent_runtime import Tools, Agent, append_trace, blocked_by_guardrails, MAX_HOPS, REQ_TIMEOUT

ROOT = Path(__file__).resolve().parents[1]
OUT_DIR = ROOT / "diffusion"
OUT_DIR.mkdir(parents=True, exist_ok=True)
CFG = json.loads((ROOT/"week7_run_config.json").read_text())

MODEL_ID = CFG["model_id"]
NEGATIVE = CFG["negative_prompt"]
SD_STEPS = int(CFG["sd_steps"])
SD_GUIDE = float(CFG["sd_guidance"])
W, H = CFG["size"]

# ------- SD pipeline (lazy) ----------
_sd_pipe = None
def get_sd():
    global _sd_pipe
    if _sd_pipe is None:
        from diffusers import StableDiffusionPipeline
        device = "cuda" if torch.cuda.is_available() else "cpu"
        _sd_pipe = StableDiffusionPipeline.from_pretrained(
            MODEL_ID,
            torch_dtype=torch.float16 if device=="cuda" else torch.float32
        )
        if device == "cuda":
            _sd_pipe.enable_attention_slicing()
            _sd_pipe.to(device)
    return _sd_pipe

def sd_generate_callable(prompt: str, negative_prompt: str, steps: int, guidance: float, height: int, width: int, filename_prefix: str = "agent") -> str:
    """Wrapper used by Tools to generate & save an image; returns filename only."""
    pipe = get_sd()
    img = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=steps or SD_STEPS,
        guidance_scale=guidance or SD_GUIDE,
        height=height or H,
        width=width or W
    ).images[0]
    fname = f"{filename_prefix}_{uuid.uuid4().hex[:8]}.png"
    img.save(OUT_DIR / fname)
    return fname

# ------- Auth & Guardrails ----------
def require_auth(authorization: str | None = Header(default=None)):
    token = settings.app_api_token
    if not token:
        return
    if not authorization or not authorization.startswith("Bearer "):
        raise HTTPException(status_code=401, detail="Missing bearer token")
    if authorization.split(" ", 1)[1].strip() != token:
        raise HTTPException(status_code=403, detail="Invalid token")

def guardrails(text: str):
    bad = blocked_by_guardrails(text)
    if bad:
        raise HTTPException(status_code=400, detail=f"Blocked by guardrails: {bad}")

# ------- Pydantic Schemas ----------
class QARequest(BaseModel):
    query: str = Field(...)

class QAResponse(BaseModel):
    answer: str
    citations: List[Dict[str,str]]
    latency_ms: int

class GenRequest(BaseModel):
    prompt: str
    negative_prompt: Optional[str] = NEGATIVE
    steps: Optional[int] = SD_STEPS
    guidance: Optional[float] = SD_GUIDE
    height: Optional[int] = H
    width: Optional[int] = W

class GenResponse(BaseModel):
    filename: str
    latency_ms: int

class AgentRequest(BaseModel):
    input: str

class AgentHop(BaseModel):
    tool: str
    input: Dict
    output_preview: str
    latency_ms: int

class AgentResponse(BaseModel):
    final: str
    hops: List[AgentHop]
    citations: List[Dict[str,str]] = []
    image_filename: Optional[str] = None
    latency_ms: int

# ------- App ----------
app = FastAPI(title="Week 7 Backend", version="2.0")

@app.get("/health")
def health():
    return {"ok": True, "model": MODEL_ID, "agent": True}

# ---- Stub QA (existing) ----
def project_qa(query: str):
    ans = f"Stubbed answer for: {query} (replace with your Graph-RAG/Multi-Hop)."
    cites = [
        {"title": "Week 7 Repo", "url": "https://github.com/sudhakarjerribanda/Capston_Handson/tree/main/Week_7"},
        {"title": "Task 1 Colab", "url": "https://colab.research.google.com/drive/1yZd5zUUmNjeDH-_mpVEkNCO4fonsmW69?usp=sharing"}
    ]
    return ans, cites

@app.post("/qa", response_model=QAResponse)
def qa(req: QARequest, _=require_auth()):
    t0 = time.time()
    guardrails(req.query)
    ans, cites = project_qa(req.query)
    return QAResponse(answer=ans, citations=cites, latency_ms=int((time.time()-t0)*1000))

@app.post("/generate", response_model=GenResponse)
def generate(req: GenRequest, _=require_auth()):
    t0 = time.time()
    guardrails(req.prompt)
    fname = sd_generate_callable(
        prompt=req.prompt,
        negative_prompt=req.negative_prompt or NEGATIVE,
        steps=req.steps or SD_STEPS,
        guidance=req.guidance or SD_GUIDE,
        height=req.height or H,
        width=req.width or W,
        filename_prefix="api"
    )
    return GenResponse(filename=fname, latency_ms=int((time.time()-t0)*1000))

# ---- Real Agent Endpoint ----
@app.post("/agent", response_model=AgentResponse)
def agent(req: AgentRequest, _=require_auth()):
    guardrails(req.input)
    # Make tools & agent
    tools = Tools(sd_callable=sd_generate_callable, negative_prompt=NEGATIVE, size=(W, H))
    agent = Agent(tools)
    deadline = time.time() + REQ_TIMEOUT

    trace = agent.run(req.input, deadline_ts=deadline)
    append_trace(ROOT, req.input, trace)

    hops = [AgentHop(tool=h["tool"], input=h["input"], output_preview=h["output_preview"], latency_ms=h["latency_ms"]) for h in trace["hops"]]
    return AgentResponse(
        final=trace["final"],
        hops=hops,
        citations=trace.get("citations", []),
        image_filename=trace.get("image"),
        latency_ms=trace.get("latency_ms", 0)
    )


Writing Week_7/backend/main.py


## B) Restart backend (same tunnel helper from Task-B)

In [6]:
# Restart the backend & re-expose (uses your existing expose_port helper if defined)
import nest_asyncio, uvicorn, threading, time, os, re, subprocess
from pathlib import Path

# Start FastAPI
nest_asyncio.apply()
def run_api():
    import Week_7.backend.main as api
    uvicorn.run("Week_7.backend.main:app", host="0.0.0.0", port=8000, log_level="info")
thread = threading.Thread(target=run_api, daemon=True); thread.start()
time.sleep(2)

# Expose port 8000 (ngrok if token, else cloudflared). If expose_port not defined, define fallback.
if "expose_port" not in globals():
    def _ensure_cloudflared():
        if not Path("cloudflared").exists():
            !curl -L https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o cloudflared -#
            !chmod +x cloudflared
    def expose_port(port: int):
        _ensure_cloudflared()
        proc = subprocess.Popen(
            ["./cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}", "--no-autoupdate"],
            stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1
        )
        public_url = None; start = time.time()
        while time.time() - start < 30:
            line = proc.stdout.readline()
            if not line: time.sleep(0.1); continue
            m = re.search(r"https://[-\w]+\.trycloudflare\.com", line.strip())
            if m: public_url = m.group(0); break
        if not public_url:
            proc.terminate(); raise RuntimeError("cloudflared did not produce a public URL in time.")
        return public_url, proc

backend_url, backend_proc = expose_port(8000)
Path("BACKEND_URL.txt").write_text(backend_url)
print("✅ Backend:", backend_url)
print("   Health:", f"{backend_url}/health")


######################################################################## 100.0%


Exception in thread Thread-3 (run_api):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-3707022144.py", line 8, in run_api
  File "/content/Week_7/backend/main.py", line 10, in <module>
    from .settings import settings
ModuleNotFoundError: No module named 'Week_7.backend.settings'


✅ Backend: https://susan-jessica-responded-proposed.trycloudflare.com
   Health: https://susan-jessica-responded-proposed.trycloudflare.com/health


## C) Five agent test queries (trace-friendly & JSON-safe)

In [7]:
import os, json, requests, textwrap
from pathlib import Path

BASE = Path("BACKEND_URL.txt").read_text().strip().rstrip("/")
headers = {"Authorization": f"Bearer {os.environ['APP_API_TOKEN']}"} if os.environ.get("APP_API_TOKEN") else {}

def post_json(path, payload, timeout=180):
    url = f"{BASE}{path}"
    r = requests.post(url, json=payload, headers=headers, timeout=timeout)
    ct = (r.headers.get("content-type") or "").lower()
    print(f"\nPOST {url} -> {r.status_code} {ct}")
    if "application/json" in ct:
        data = r.json()
        print(json.dumps(data, indent=2)[:600], "...")
        return data
    else:
        print(textwrap.shorten(r.text.replace("\n"," "), width=400))
        return None

tests = [
    {"input": "Generate an infographic of a trust game with BDI panels and reciprocity gauge"},
    {"input": "Explain Graph-RAG evidence integration for IMRaD sections"},
    {"input": "calc: (1+2*3)^2 / 7"},
    {"input": "CSV: colA,colB\n1,10\n2,20\n3,30"},
    {"input": "Create a clean isometric diagram of recruiter → MDT → ICT triage flow"}
]

results = []
for t in tests:
    data = post_json("/agent", t, timeout=240)
    results.append(data)
print("\n✅ Ran 5 agent queries.")



POST https://susan-jessica-responded-proposed.trycloudflare.com/agent -> 502 text/plain; charset=utf-8
502 Bad Gateway Unable to reach the origin service. The service may be down or it may not be responding to traffic from cloudflared

POST https://susan-jessica-responded-proposed.trycloudflare.com/agent -> 502 text/plain; charset=utf-8
502 Bad Gateway Unable to reach the origin service. The service may be down or it may not be responding to traffic from cloudflared

POST https://susan-jessica-responded-proposed.trycloudflare.com/agent -> 502 text/plain; charset=utf-8
502 Bad Gateway Unable to reach the origin service. The service may be down or it may not be responding to traffic from cloudflared

POST https://susan-jessica-responded-proposed.trycloudflare.com/agent -> 502 text/plain; charset=utf-8
502 Bad Gateway Unable to reach the origin service. The service may be down or it may not be responding to traffic from cloudflared

POST https://susan-jessica-responded-proposed.trycloudf